# AUTOMATING LINKEDIN NETWORKING

In [ ]:
#Loading user credentials to log in

with open('private/user.txt','r') as f: 
    user, password = f.read().split('\n')

### Importing required libraries:

In [ ]:
# Selenium:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options #Chromedriver optiones
from selenium.webdriver.support.ui import WebDriverWait #Set wait time
from selenium.webdriver.support import expected_conditions as EC #Wait for element to load
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains as AC


#BeautifulSoup:

import requests as req
from bs4 import BeautifulSoup as bs
from selenium.webdriver.chrome.service import Service

#Miscellaneous:
import time 
import regex as re

#Chrome driver options:


service = Service(executable_path=PATH) #Changed since Selenium 4.10.0
opciones=Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#opciones.add_argument('--disable-blink-features=AutomationControlled')
opciones.add_experimental_option('useAutomationExtension', False)
#opciones.headless=False    
opciones.add_argument('--start-maximized')
opciones.add_argument('--incognito')               # incognito mode

########################################################################################################################
#usuario=UserAgent().random
#opciones.add_argument(f'user-agent={usuario}')

#opciones.add_argument("window-size=1366,768") # tamaño de la ventana
#opciones.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36") #Navegador
#opciones.add_extension('driver/adblock.crx')       # adblocker#opciones.add_argument('--incognito')
#opciones.add_extension('driver_folder/adblock.crx')
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies

In [ ]:
PATH=ChromeDriverManager().install() #Installing last version of chromedriver

URL='https://www.linkedin.com' #URL to login

In [ ]:
# Initializing  Chrome driver & options
driver = webdriver.Chrome(service=service, options=opciones)

#driver=webdriver.Chrome(PATH, options=opciones) #Changed since Selenium 4.10.0

### Browsing LinkedIn:

In [ ]:
# Loading LinkedIn:
driver.get(URL)
time.sleep(1.5)

# Closing cookies warning:
driver.find_element(By.XPATH, '//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[2]').click()

#Filling User:
time.sleep(1.5)
driver.find_element(By.XPATH, '//*[@id="session_key"]').send_keys(user)


#Filling Password:
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="session_password"]').send_keys(password)

time.sleep(1)
driver.find_element(By.CLASS_NAME, 'sign-in-form__submit-button').click()

#Closin messages tab: (In case your linkedin account has a message tab, otherwise comment this block)
messages_tab = WebDriverWait(driver,4).until( #Max wait 4 secs
                EC.presence_of_element_located((By.XPATH,
                '/html/body/div[7]/aside/div[1]/header/div[3]/button[2]'))) #Wait for element to load
messages_tab.click()


network_button = WebDriverWait(driver, 10).until( #Max wait 10 secs
                EC.presence_of_element_located((By.ID, 'ember18'))) #Wait for element to load
network_button.click()

#Waiting for 'More suggestions block' to load and browsing to the end of the page:
suggest_section = WebDriverWait(driver, 12).until( #Max wait 10 secs
                EC.presence_of_element_located((By.CLASS_NAME, 'pb2'))) #Wait for element to load

#Scroll and let page load 10 times:
for scroll in range(0,15):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1.5)
    if not scroll:
        first_contact = driver.execute_script("return document.body.scrollHeight") #Get inital page height, to go backto it later

time.sleep(1)
  
#Go back to the beginning of contacts:
target = driver.find_element(By.CLASS_NAME, 'mn-discovery__header') #Point we want to scroll to
target.location_once_scrolled_into_view


#Beatiful Soup:
soup = bs(driver.page_source, 'html.parser') #Loading page source
contacts_seccion = soup.find('section',class_='pb2') #Section were more suggestions are located
contacts_list = contacts_seccion.find_all('li') #List of contacts
close_buttons = contacts_seccion.find_all('button',class_='artdeco-card__dismiss') #List of closing buttons
connect_button = contacts_seccion.find_all('button', class_='artdeco-button--2') #List of connect buttons


#List of contacts loaded, now we need to connect or dismiss them:

add_counter = 0 #Counter for added contacts
dismiss_counter = 0 #Counter for dismissed contacts

for contact in range(0,len(contacts_list)): #Iterating over contacts list
    try :
        commons_string = contacts_list[contact].find('span',class_='member-insights__reason').text #String of common connections
        common_contacts_num = int(re.search(r'\d+', commons_string).group()) #Apply regex to get number of common connections
        
    except:
        common_contacts_num = 0 #If there are no common connections, set number to 0
    
    if common_contacts_num > 50: #If common connections are more than 15, add to list
        add_button_id = connect_button[contact].get('id') #Get element id
        driver.find_element(By.ID, add_button_id).click()
        close_button_id = close_buttons[contact].get('id') #Get element id
        driver.find_element(By.ID, close_button_id).click()
        add_counter += 1 #Add to counter
        time.sleep(2)
        
    else:
        target_item_id = close_buttons[contact].get('id') #Get element id
        driver.find_element(By.ID, target_item_id).click()
        dismiss_counter += 1 #Add to counter
        time.sleep(2)
print(f'Added {add_counter} contacts and dismissed {dismiss_counter} contacts\n -->TOTAL: {add_counter+dismiss_counter}')
driver.quit()